In [33]:
import joblib
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
import random

warnings.filterwarnings('ignore')

In [2]:
# vect = joblib.load("tfidf.pkl")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [40]:
# class QADataset: 
#     """Question and answers dataset""" 
     
#     def __init__(self, path, lim): 
#         """ 
#         Args: 
#             path (string): Path to jsonl file 
#             ]
#         """ 
#         self.path = path 
#         data = [] 
#         self.text = [] 
#         with open(path, 'r') as json_file: 
#             json_list = list(json_file) 
#             for json_str in tqdm(json_list[:lim] if lim != -1 else json_list): 
#                 item = json.loads(json_str) 
#                 text = item['passage']['text'] 
#                 text_vect = vect.transform([re.sub(r'(@\w+)|(\\n)', " ", text)])
#                 self.text.append(text) 
#                 entities = item['passage']['entities'] 
#                 for row in item['qas']: 
#                     for ans in row.get('answers', [None]): 
#                         q_text = row['query'] 
#                         for i, en in enumerate(entities): 
#                             word = text[en['start']:en['end']] 
#                             n_q = q_text.replace('@placeholder', word) 
#                             data.append({
#                                             "idx": f'{item["idx"]}/{i}', 
#                                             "cosine": cosine_similarity(text_vect, vect.transform([n_q]))[0][0]
#                                         })
                            
#                             if not ans:
#                                 continue 
 
#                             data[-1]["label"] = int(ans['text'].strip() == word.strip())
    
#                 self.text[-1] = re.sub(r'(@\w+)|(\\n)', " ", self.text[-1])
#         self.data = pd.DataFrame.from_dict(data).set_index("idx")


In [41]:
# train_df = QADataset('data/rucos_train.jsonl', lim=8000).data
# val_df = QADataset('data/rucos_val.jsonl', lim=-1).data
# test_df = QADataset('data/rucos_test.jsonl', lim=-1).data

100%|██████████| 7256/7256 [03:37<00:00, 33.42it/s]


In [56]:
# train_df.to_pickle("data/train_cos.pkl")
# val_df.to_pickle("data/val_cos.pkl")
# test_df.to_pickle("data/test_cos.pkl")

# Load data

In [57]:
train_df = pd.read_pickle("data/train_cos.pkl")
val_df = pd.read_pickle("data/val_cos.pkl")
test_df = pd.read_pickle("data/test_cos.pkl")

In [58]:
train_df

,cosine,label
idx,,
0/0,0.030596,0
0/1,0.030596,0
0/2,0.030596,0
0/3,0.030596,0
0/4,0.055020,0
...,...,...
7999/3,0.176504,0
7999/4,0.232124,0
7999/5,0.238499,0


In [120]:
Counter(train_df.label)

Counter({0: 266664, 1: 45098})

In [130]:
list(dict(Counter(train_df.label)).values())[::-1]

[45098, 266664]

# Catboost

In [ ]:
# !pip install catboost

In [131]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
                            learning_rate=1e-2,
                            max_depth=4,
                            iterations=1000,
                            loss_function="MultiClass",
                            eval_metric='MultiClass',
                            random_seed=179,
                            class_weights=list(dict(Counter(train_df.label)).values())[::-1],
#                            task_type='GPU'
                          )

In [132]:
model.fit(train_df.drop(['label'], axis=1), train_df.label)

0:	learn: 0.6930748	total: 52.6ms	remaining: 52.5s
1:	learn: 0.6928847	total: 108ms	remaining: 53.7s
2:	learn: 0.6926717	total: 178ms	remaining: 59.1s
3:	learn: 0.6924649	total: 234ms	remaining: 58.2s
4:	learn: 0.6922721	total: 302ms	remaining: 1m
5:	learn: 0.6920766	total: 356ms	remaining: 59s
6:	learn: 0.6918971	total: 411ms	remaining: 58.3s
7:	learn: 0.6917160	total: 473ms	remaining: 58.6s
8:	learn: 0.6915438	total: 549ms	remaining: 1m
9:	learn: 0.6913639	total: 602ms	remaining: 59.6s
10:	learn: 0.6911993	total: 661ms	remaining: 59.4s
11:	learn: 0.6910122	total: 722ms	remaining: 59.5s
12:	learn: 0.6908764	total: 800ms	remaining: 1m
13:	learn: 0.6907132	total: 859ms	remaining: 1m
14:	learn: 0.6905878	total: 913ms	remaining: 60s
15:	learn: 0.6904248	total: 979ms	remaining: 1m
16:	learn: 0.6902658	total: 1.06s	remaining: 1m 1s
17:	learn: 0.6901232	total: 1.14s	remaining: 1m 2s
18:	learn: 0.6899763	total: 1.21s	remaining: 1m 2s
19:	learn: 0.6898789	total: 1.27s	remaining: 1m 2s
20:	lear

# Make submition

In [133]:
def save_pred(filename, preds):
    with open('data/rucos_test.jsonl', 'r') as json_file:
        test = list(map(lambda x: json.loads(x), list(json_file)))
    ind = 0
    res = []
    for i in range(len(test)):
        ent_prob = []
        for j in range(len(test[i]["passage"]['entities'])):
            ent_prob.append(preds[ind] if ind < len(preds) else 0)
            ind += 1
        ans_ind = np.array(ent_prob).argmax()
        s, f = test[i]["passage"]["entities"][ans_ind]['start'], test[i]["passage"]["entities"][ans_ind]['end']
        res.append({"idx": test[i]["idx"], "text": test[i]["passage"]['text'][s:f]})
    
    !mkdir submitions/
    
    with jsonlines.open("submitions/" + filename, 'w') as file:
        file.write_all(res)

In [134]:
pred_proba = model.predict_proba(test_df)

In [135]:
pred_proba = [elem[1] for elem in pred_proba]

In [140]:
cnt = Counter([int(elem >= .5) for elem in pred_proba])

In [96]:
len(test_df), len(pred_proba)

(96986, 96986)

In [137]:
file_name = f'{__import__("datetime").datetime.now()}.jsonl'
save_pred(file_name, pred_proba)

mkdir: submitions/: File exists


# Send to tg

In [138]:
#@title Telegram Logger (doc sender) { run: "auto", vertical-output: true, display-mode: "form" }
token = "1282076787:AAEC3rYsSHh_livmutl0qDEemVhlhjWpWyI" #@param {type:"string"}
user_id =  -584355986 #@param {type:"integer"}
combine = True #@param ["True", "False"] {type:"raw"}

import telebot
bot = telebot.TeleBot(token)
last_message = None

def logger(txt):
    global last_message
    while True:
        try:
            last_message_text = last_message.text if last_message != None else ''

            if type(txt) is tqdm:
                txt = txt.format_dict

                rate = txt["rate"] if txt["rate"] else 0
                remain = int((txt['total'] - txt['n']) / rate) if rate != 0 else 0

                txt = f"{txt['prefix']} - {int(txt['n'] / txt['total'] * 100)}%, {txt['n']}/{txt['total']} [{str(int(txt['elapsed']) // 60).rjust(2, '0')}:{str(int(txt['elapsed']) % 60).rjust(2, '0')}<{str(remain // 60).rjust(2, '0')}:{str(remain % 60).rjust(2, '0')}]"
                
                last_message_text = '\n'.join(last_message_text.split('\n')[:-1])

            if not last_message or not combine:
                last_message = bot.send_message(user_id, txt)
            else:
                last_message = bot.edit_message_text(chat_id=user_id,
                                      message_id=last_message.message_id,
                                      text=last_message_text + '\n' + txt)
            return
        except Exception as ex:
            print("Exception while logging:")
            print(ex)
            pass

In [142]:
while True:
    try:
        bot.send_document(user_id, 
                          open('submitions/' + file_name, 'rb'),
                          caption=f'{cnt}\nLabel 0: {cnt[0] / len(pred_proba) * 100:.3f}%\nLabel 1: {cnt[1] / len(pred_proba) * 100:.3f}%')
        break
    except:
        import time
        time.sleep(1)